# Reconciliation #01: Balance Sheet Identity

## Overview
- **What this notebook tests:** The fundamental accounting identity `Assets = Liabilities + Equity` holds at every time step across a multi-year simulation, and that retained earnings correctly accumulate net income minus dividends.
- **Prerequisites:** `pip install -e .` from the project root.
- **Estimated runtime:** < 60 seconds
- **Audience:** Developers, actuaries, and auditors verifying accounting integrity.

## Checks Performed
1. **Accounting Identity:** `Total Assets == Total Liabilities + Total Equity` at each year-end (tolerance < $0.01)
2. **Retained Earnings Accumulation:** `RE[t] == RE[t-1] + Net Income[t] - Dividends[t]`
3. **Accumulated Depreciation Consistency:** Accumulated depreciation grows consistently with depreciation expense
4. **Ledger Trial Balance:** Total debits == total credits (the ledger is balanced)

## Approach
We create a `WidgetManufacturer` with known initial conditions ($10M assets, 10% margin, 25% tax rate) and run a deterministic 10-year simulation with zero growth rate (no stochastic shocks). At each year-end we extract the full balance sheet from the ledger and verify the identities above.

In [ ]:
"""Setup: imports and notebook header."""
import sys
import os
import warnings
from decimal import Decimal

import numpy as np
import pandas as pd
from IPython.display import display, HTML

# Reconciliation helper utilities
sys.path.insert(0, os.path.dirname(os.path.abspath(".")))
from _reconciliation_helpers import (
    ReconciliationChecker, final_summary, section_header,
    notebook_header, timed_cell, fmt_dollar, display_df,
    create_standard_manufacturer,
)

# Core framework imports
from ergodic_insurance.config import ManufacturerConfig
from ergodic_insurance.manufacturer import WidgetManufacturer
from ergodic_insurance.ledger import AccountName, Ledger

# Display the standard notebook header
notebook_header(
    number=1,
    title="Balance Sheet Identity",
    description=(
        "Verifies that Assets = Liabilities + Equity at every time step, "
        "retained earnings accumulate correctly, and the ledger trial balance sums to zero."
    ),
)

# Suppress verbose logging from the manufacturer during simulation
import logging
logging.getLogger("ergodic_insurance").setLevel(logging.WARNING)

print("Setup complete.")

## Section 1: Create Manufacturer and Run Deterministic Simulation

We initialize a `WidgetManufacturer` with standard parameters and run a 10-year
deterministic simulation (zero growth, no stochastic shocks, fixed seed) to create
a clean, reproducible set of financial statements to audit.

In [ ]:
"""Run deterministic 10-year simulation."""
section_header("1. Deterministic 10-Year Simulation")

N_YEARS = 10
TOLERANCE = Decimal("0.01")  # $0.01 tolerance for accounting identity

with timed_cell("Simulation"):
    # Create manufacturer with standard parameters
    mfr = create_standard_manufacturer(
        initial_assets=10_000_000,
        asset_turnover=1.2,
        operating_margin=0.10,
        tax_rate=0.25,
        retention_ratio=0.70,
    )

    # Capture initial state (year 0, before any steps)
    initial_state = {
        "total_assets": mfr.total_assets,
        "total_liabilities": mfr.total_liabilities,
        "equity": mfr.equity,
        "retained_earnings": mfr.ledger.get_balance(AccountName.RETAINED_EARNINGS),
        "accumulated_depreciation": mfr.accumulated_depreciation,
    }

    # Run simulation: zero growth, no stochastic shocks
    all_metrics = []
    for year in range(N_YEARS):
        metrics = mfr.step(
            letter_of_credit_rate=0.015,
            growth_rate=0.0,
            time_resolution="annual",
            apply_stochastic=False,
        )
        all_metrics.append(metrics)

print(f"Simulation complete: {N_YEARS} years")
print(f"Initial total assets: {fmt_dollar(initial_state['total_assets'])}")
print(f"Final total assets:   {fmt_dollar(mfr.total_assets)}")
print(f"Final equity:         {fmt_dollar(mfr.equity)}")

## Section 2: Accounting Identity Check (A = L + E)

At each year-end, verify that `Total Assets == Total Liabilities + Total Equity`.
The manufacturer computes equity as `total_assets - total_liabilities`, so this
also validates that the ledger-derived balance sheet components are internally
consistent.

In [ ]:
"""Check A = L + E at every year-end."""
section_header("2. Accounting Identity: Assets = Liabilities + Equity")

checker_identity = ReconciliationChecker(section="Accounting Identity")

with timed_cell("Identity checks"):
    # Re-run simulation to capture year-end snapshots
    mfr2 = create_standard_manufacturer(
        initial_assets=10_000_000,
        asset_turnover=1.2,
        operating_margin=0.10,
        tax_rate=0.25,
        retention_ratio=0.70,
    )

    identity_rows = []

    # Check initial state (year 0)
    ta = mfr2.total_assets
    tl = mfr2.total_liabilities
    eq = mfr2.equity
    diff = abs(ta - (tl + eq))
    identity_rows.append({
        "Year": 0,
        "Total Assets": float(ta),
        "Total Liabilities": float(tl),
        "Equity": float(eq),
        "L + E": float(tl + eq),
        "Difference": float(diff),
        "Status": "PASS" if diff <= TOLERANCE else "FAIL",
    })
    checker_identity.assert_close(
        float(ta), float(tl + eq), tol=float(TOLERANCE),
        message=f"Year 0: A = L + E",
        label_actual="Assets", label_expected="L + E",
    )

    # Run each year and check
    for year in range(1, N_YEARS + 1):
        mfr2.step(
            letter_of_credit_rate=0.015,
            growth_rate=0.0,
            time_resolution="annual",
            apply_stochastic=False,
        )
        ta = mfr2.total_assets
        tl = mfr2.total_liabilities
        eq = mfr2.equity
        diff = abs(ta - (tl + eq))

        identity_rows.append({
            "Year": year,
            "Total Assets": float(ta),
            "Total Liabilities": float(tl),
            "Equity": float(eq),
            "L + E": float(tl + eq),
            "Difference": float(diff),
            "Status": "PASS" if diff <= TOLERANCE else "FAIL",
        })
        checker_identity.assert_close(
            float(ta), float(tl + eq), tol=float(TOLERANCE),
            message=f"Year {year}: A = L + E",
            label_actual="Assets", label_expected="L + E",
        )

# Display year-by-year results table
df_identity = pd.DataFrame(identity_rows)
df_identity["Total Assets"] = df_identity["Total Assets"].map(lambda x: f"${x:,.2f}")
df_identity["Total Liabilities"] = df_identity["Total Liabilities"].map(lambda x: f"${x:,.2f}")
df_identity["Equity"] = df_identity["Equity"].map(lambda x: f"${x:,.2f}")
df_identity["L + E"] = df_identity["L + E"].map(lambda x: f"${x:,.2f}")
df_identity["Difference"] = df_identity["Difference"].map(lambda x: f"${x:,.4f}")
display_df(df_identity, title="Year-by-Year Accounting Identity Check")

checker_identity.display_results()

## Section 3: Retained Earnings Accumulation

Verify that retained earnings at each year-end equal the prior year's retained
earnings plus net income minus dividends paid:

```
RE[t] = RE[t-1] + Net Income[t] - Dividends[t]
```

We track retained earnings directly from the ledger (the single source of truth)
and compare against the net income and dividends recorded in the metrics history.

In [ ]:
"""Check retained earnings accumulation: RE[t] = RE[t-1] + NI[t] - Div[t]."""
section_header("3. Retained Earnings Accumulation")

checker_re = ReconciliationChecker(section="Retained Earnings")

with timed_cell("RE accumulation checks"):
    # Re-run simulation capturing RE at each step
    mfr3 = create_standard_manufacturer(
        initial_assets=10_000_000,
        asset_turnover=1.2,
        operating_margin=0.10,
        tax_rate=0.25,
        retention_ratio=0.70,
    )

    re_rows = []
    prev_re = mfr3.ledger.get_balance(AccountName.RETAINED_EARNINGS)

    for year in range(1, N_YEARS + 1):
        metrics = mfr3.step(
            letter_of_credit_rate=0.015,
            growth_rate=0.0,
            time_resolution="annual",
            apply_stochastic=False,
        )

        # Get current retained earnings from ledger
        current_re = mfr3.ledger.get_balance(AccountName.RETAINED_EARNINGS)
        net_income = metrics["net_income"]
        dividends = metrics["dividends_paid"]

        # Expected RE = prior RE + net income - dividends
        expected_re = prev_re + net_income - dividends
        diff = abs(current_re - expected_re)

        re_rows.append({
            "Year": year,
            "RE (prior)": float(prev_re),
            "Net Income": float(net_income),
            "Dividends": float(dividends),
            "Expected RE": float(expected_re),
            "Actual RE": float(current_re),
            "Difference": float(diff),
            "Status": "PASS" if diff <= float(TOLERANCE) else "FAIL",
        })

        checker_re.assert_close(
            float(current_re), float(expected_re), tol=float(TOLERANCE),
            message=f"Year {year}: RE[t] = RE[t-1] + NI - Div",
            label_actual="Actual RE", label_expected="Expected RE",
        )

        prev_re = current_re

# Display retained earnings table
df_re = pd.DataFrame(re_rows)
for col in ["RE (prior)", "Net Income", "Dividends", "Expected RE", "Actual RE"]:
    df_re[col] = df_re[col].map(lambda x: f"${x:,.2f}")
df_re["Difference"] = df_re["Difference"].map(lambda x: f"${x:,.4f}")
display_df(df_re, title="Year-by-Year Retained Earnings Accumulation")

checker_re.display_results()

## Section 4: Accumulated Depreciation Consistency

Verify that accumulated depreciation grows consistently with the depreciation
expense recorded each period. Since we use straight-line depreciation on PP&E,
we expect:

```
Accum Depr[t] = Accum Depr[t-1] + Depreciation Expense[t]
```

We track `accumulated_depreciation` from the ledger (contra-asset) and the
per-period depreciation expense from the `DEPRECIATION_EXPENSE` account.

In [ ]:
"""Check accumulated depreciation consistency."""
section_header("4. Accumulated Depreciation Consistency")

checker_depr = ReconciliationChecker(section="Depreciation")

with timed_cell("Depreciation checks"):
    # Re-run simulation capturing depreciation at each step
    mfr4 = create_standard_manufacturer(
        initial_assets=10_000_000,
        asset_turnover=1.2,
        operating_margin=0.10,
        tax_rate=0.25,
        retention_ratio=0.70,
    )

    depr_rows = []
    prev_accum_depr = mfr4.accumulated_depreciation  # Should be 0 at start

    for year in range(1, N_YEARS + 1):
        # Capture gross PPE before step (depreciation is based on gross PPE at start of period)
        gross_ppe_before = mfr4.gross_ppe

        metrics = mfr4.step(
            letter_of_credit_rate=0.015,
            growth_rate=0.0,
            time_resolution="annual",
            apply_stochastic=False,
        )

        current_accum_depr = mfr4.accumulated_depreciation
        # The period depreciation expense is the change in accumulated depreciation
        period_depr = current_accum_depr - prev_accum_depr

        # Verify accumulated depreciation increased by a positive amount
        # (since we have PP&E with remaining net book value)
        depr_rows.append({
            "Year": year,
            "Gross PPE (start)": float(gross_ppe_before),
            "Period Depreciation": float(period_depr),
            "Accum Depr (prior)": float(prev_accum_depr),
            "Accum Depr (current)": float(current_accum_depr),
            "Net PPE": float(mfr4.net_ppe),
        })

        # Check: accumulated depreciation increased by a non-negative amount
        checker_depr.check(
            period_depr >= Decimal("0"),
            message=f"Year {year}: Depreciation expense >= 0",
            detail=f"Period depreciation = {fmt_dollar(period_depr)}",
        )

        # Check: accumulated depreciation is additive
        expected_accum = prev_accum_depr + period_depr
        checker_depr.assert_close(
            float(current_accum_depr), float(expected_accum), tol=float(TOLERANCE),
            message=f"Year {year}: Accum Depr[t] = Accum Depr[t-1] + Depr Exp[t]",
            label_actual="Actual Accum Depr", label_expected="Expected Accum Depr",
        )

        # Check: net PPE = gross PPE - accumulated depreciation
        expected_net_ppe = mfr4.gross_ppe - current_accum_depr
        checker_depr.assert_close(
            float(mfr4.net_ppe), float(expected_net_ppe), tol=float(TOLERANCE),
            message=f"Year {year}: Net PPE = Gross PPE - Accum Depr",
            label_actual="Net PPE", label_expected="Gross - Accum",
        )

        prev_accum_depr = current_accum_depr

# Display depreciation table
df_depr = pd.DataFrame(depr_rows)
for col in ["Gross PPE (start)", "Period Depreciation", "Accum Depr (prior)",
            "Accum Depr (current)", "Net PPE"]:
    df_depr[col] = df_depr[col].map(lambda x: f"${x:,.2f}")
display_df(df_depr, title="Year-by-Year Depreciation Schedule")

checker_depr.display_results()

## Section 5: Ledger Trial Balance (Debits == Credits)

The double-entry bookkeeping invariant requires that total debits always equal
total credits. We verify this at each year-end using the ledger's built-in
`verify_balance()` method, which sums all debit and credit entries.

In [ ]:
"""Check ledger trial balance: debits == credits."""
section_header("5. Ledger Trial Balance (Debits == Credits)")

checker_tb = ReconciliationChecker(section="Trial Balance")

with timed_cell("Trial balance checks"):
    # Re-run simulation and check trial balance at each year-end
    mfr5 = create_standard_manufacturer(
        initial_assets=10_000_000,
        asset_turnover=1.2,
        operating_margin=0.10,
        tax_rate=0.25,
        retention_ratio=0.70,
    )

    tb_rows = []

    # Check initial state
    is_balanced, difference = mfr5.ledger.verify_balance()
    tb_rows.append({
        "Year": 0,
        "Balanced": is_balanced,
        "Difference": float(difference),
        "Num Entries": len(mfr5.ledger.entries),
        "Status": "PASS" if is_balanced else "FAIL",
    })
    checker_tb.check(
        is_balanced,
        message="Year 0: Debits == Credits",
        detail=f"Difference = ${float(difference):,.4f}, Entries = {len(mfr5.ledger.entries)}",
    )

    for year in range(1, N_YEARS + 1):
        mfr5.step(
            letter_of_credit_rate=0.015,
            growth_rate=0.0,
            time_resolution="annual",
            apply_stochastic=False,
        )

        is_balanced, difference = mfr5.ledger.verify_balance()
        tb_rows.append({
            "Year": year,
            "Balanced": is_balanced,
            "Difference": float(difference),
            "Num Entries": len(mfr5.ledger.entries),
            "Status": "PASS" if is_balanced else "FAIL",
        })
        checker_tb.check(
            is_balanced,
            message=f"Year {year}: Debits == Credits",
            detail=f"Difference = ${float(difference):,.4f}, Entries = {len(mfr5.ledger.entries)}",
        )

    # Also check that the trial balance has non-zero accounts
    trial = mfr5.ledger.get_trial_balance()
    checker_tb.check(
        len(trial) > 0,
        message="Trial balance has non-zero accounts",
        detail=f"{len(trial)} accounts with non-zero balances",
    )

# Display trial balance results
df_tb = pd.DataFrame(tb_rows)
df_tb["Difference"] = df_tb["Difference"].map(lambda x: f"${x:,.4f}")
display_df(df_tb, title="Year-by-Year Trial Balance Check")

# Show final trial balance detail
print("\nFinal Trial Balance (Year 10):")
print("-" * 50)
for account, balance in sorted(trial.items()):
    print(f"  {account:40s} {fmt_dollar(balance):>15s}")

checker_tb.display_results()

## Section 6: Combined Year-by-Year Summary

A consolidated view of all four identity checks across all simulation years,
providing a single dashboard to confirm the integrity of the accounting model.

In [ ]:
"""Combined summary of all checks across all years."""
section_header("6. Combined Year-by-Year Summary")

# Gather pass/fail counts
sections = [
    ("Accounting Identity (A=L+E)", checker_identity),
    ("Retained Earnings (RE accumulation)", checker_re),
    ("Depreciation Consistency", checker_depr),
    ("Trial Balance (Dr=Cr)", checker_tb),
]

summary_rows = []
for name, checker in sections:
    passed, failed = checker.summary_counts
    total = passed + failed
    summary_rows.append({
        "Check Section": name,
        "Passed": passed,
        "Failed": failed,
        "Total": total,
        "Status": "ALL PASS" if failed == 0 else f"{failed} FAILED",
    })

df_summary = pd.DataFrame(summary_rows)
display_df(df_summary, title="Reconciliation Check Summary")

# Print totals
total_passed = sum(r["Passed"] for r in summary_rows)
total_failed = sum(r["Failed"] for r in summary_rows)
total_checks = total_passed + total_failed
print(f"\nTotal: {total_passed}/{total_checks} checks passed")
if total_failed == 0:
    print("All balance sheet identity checks PASSED.")
else:
    print(f"WARNING: {total_failed} check(s) FAILED.")

## Final Result

The cell below produces the final PASS/FAIL banner. If any check failed,
it raises an `AssertionError` so that `nbconvert` or CI will catch the failure.

In [ ]:
"""Final PASS/FAIL summary — raises AssertionError on failure."""
final_summary(checker_identity, checker_re, checker_depr, checker_tb)